In [1]:
# import library
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
# url target
url = 'https://boston.craigslist.org/search/npo'

In [5]:
# Create a dictionary
d = {'key':'value'}
print(d)


# Update the dictionary
d['new key'] = 'new value'
print(d)

{'key': 'value'}
{'key': 'value', 'new key': 'new value'}


In [6]:
# loop per page
npo_jobs = {}
job_no = 0
while True:
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data,'html.parser')
    jobs = soup.find_all('p',{'class':'result-info'})
    
    # loop jobs element
    for job in jobs:
        print('Job Item = ',job)
        title = job.find('a',{'class':'result-titile'}).text
        location_tag = job.find('span',{'class':'result-hood'})
        location = location_tag.text[2:1] if location_tag else "N/A"
        date = job.find('time',{'class':'result-date'}).text
        link = job.find('a',{'class':'result-title'}).get('href')

        job_response = requests.get(link)
        job_data = job_response.text
        job_soup = BeautifuSoup(job_data,'html.parser')
        job_description = job_soup.find('section',{'id':'postingbody'}).text
        job_attributes_tag = job_soup.find('p',{'class':'attrgroup'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
        
        # counter job number of page
        job_no+=1
        npo_jobs[job_no] = [title, location, date, link, job_attributes, job_description]
        

        print('Job Title : ',title,'\nLocation : ',location,'\nDate : ',date,'\nLink : ',link,'\n',job_attributes,'\nJob Description : ',job_description,'\n---')
    
    url_tag = soup.find('a',{'title','next page'})
    if url_tag.get('href'):
        url = 'https://boston.craigslist.org'+url_tag.get('href')
        print('Next Page URL = ',url)
    else:
        break

print('Job total = ',job_no)
        

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
print("Total Jobs:", job_no)
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = 'index', columns = ['Job Title','Location','Date', 'Link', 'Job Attributes', 'Job Description'])


npo_jobs_df.head()


npo_jobs_df.to_csv('npo_jobs.csv')